In [18]:
import pandas as pd
import numpy as np

# 1. 시계열/통계 데이터 (df_time)
np.random.seed(42)
dates = pd.date_range('2023-01-01', periods=100, freq='D')
prices = np.random.lognormal(mean=2, sigma=1, size=100) * 1000 # 왜도가 있는 데이터 생성
sales = np.random.randint(10, 100, size=100).astype(float)

df_time = pd.DataFrame({'Date': dates, 'Price': prices, 'Sales': sales})

# 결측치 강제 생성 (보간법 연습용)
df_time.loc[10:20, 'Sales'] = np.nan 
df_time.loc[50:55, 'Sales'] = np.nan

# 2. 넓은 형태 데이터 (df_wide) - Melt 연습용
data_wide = {
    'Product': ['A', 'B', 'C'],
    '2023-Q1': [100, 150, 200],
    '2023-Q2': [110, 160, 210],
    '2023-Q3': [105, 155, 205],
    '2023-Q4': [120, 170, 220]
}
df = pd.DataFrame(data_wide)

print("데이터 생성 완료!")
print(df_time.head())
print(df.head())

데이터 생성 완료!
        Date         Price  Sales
0 2023-01-01  12142.529844   18.0
1 2023-01-02   6434.896126   71.0
2 2023-01-03  14121.359907   46.0
3 2023-01-04  33886.945659   60.0
4 2023-01-05   5846.520075   53.0
  Product  2023-Q1  2023-Q2  2023-Q3  2023-Q4
0       A      100      110      105      120
1       B      150      160      155      170
2       C      200      210      205      220


In [19]:
skew_orig = df_time['Price'].skew()
#print(skew_orig)

df_time['Price_Log'] = np.log1p(df_time['Price'])
skew_log = df_time['Price_Log'].skew()
result1 = abs(skew_orig - skew_log)
print(round(result1, 2))

2.07


In [20]:
df_time['Sales_fill'] = df_time['Sales'].interpolate(method='linear')
df_time['Prev_Sales'] = df_time['Sales_fill'].shift(1)
result2 = df_time['Sales_fill'].corr(df_time['Prev_Sales'])
print(round(result2, 2))
print(df_time.head())

0.25
        Date         Price  Sales  Price_Log  Sales_fill  Prev_Sales
0 2023-01-01  12142.529844   18.0   9.404552        18.0         NaN
1 2023-01-02   6434.896126   71.0   8.769646        71.0        18.0
2 2023-01-03  14121.359907   46.0   9.555515        46.0        71.0
3 2023-01-04  33886.945659   60.0  10.430815        60.0        46.0
4 2023-01-05   5846.520075   53.0   8.673773        53.0        60.0


In [21]:
#print(df.head())
df_melt = pd.melt(df, id_vars=['Product'], var_name='Quarter', value_name='Revenue')
#print(df_melt)

result3 = df_melt[df_melt['Quarter']=='2023-Q4']['Revenue'].sum()
print(round(result3))

510


In [22]:
min_val = df_time['Price'].min()
max_val = df_time['Price'].max()
df_time['Price_Scaled'] = (df_time['Price'] - min_val) / (max_val - min_val)
result4 = sum(df_time['Price_Scaled'] > 0.5)
print(result4)

8


📝 실전 연습 문제
문제 1. (조건 필터링 & 중앙값 대치) Price 컬럼의 결측치(NaN)를 **City별 Price의 중앙값(median)**으로 채우시오. 그 후, Payment가 'Card'인 데이터들의 Price 평균값을 구하시오. (단, 소수점 셋째 자리에서 반올림하여 둘째 자리까지 출력하시오.)

문제 2. (시계열 & Groupby) Date 컬럼에서 '요일'(Monday=0, ... Sunday=6) 정보를 추출하여 Weekday 컬럼을 만드시오. **'주말' (토요일, 일요일)**에 해당하는 데이터만 필터링한 후, 이 데이터들의 City별 Price **합계(sum)**를 구했을 때 가장 큰 값을 구하시오. (단, 정수로 출력하시오.)

문제 3. (데이터 구간화 & 빈도수) Price 컬럼 값(결측치 처리 완료된 상태 가정)을 기준으로 데이터를 3개의 구간으로 나눕니다.

0 이상 4000 미만: 'Low'

4000 이상 7000 미만: 'Mid'

7000 이상: 'High'

'Mid' 그룹에 속하면서 Product가 'A'인 데이터의 **개수(행의 수)**를 구하시오.

In [23]:
import pandas as pd
import numpy as np

np.random.seed(2024)

# 데이터 생성
dates = pd.date_range('2023-01-01', periods=500, freq='h')
cities = np.random.choice(['Seoul', 'Busan', 'Daegu', 'Incheon'], 500)
products = np.random.choice(['A', 'B', 'C'], 500)
prices = np.random.randint(1000, 10000, 500)
payments = np.random.choice(['Card', 'Cash', 'Pay'], 500)

df = pd.DataFrame({
    'Date': dates,
    'City': cities,
    'Product': products,
    'Price': prices,
    'Payment': payments
})

# 결측치 임의 생성 (Price 컬럼)
df.loc[10:20, 'Price'] = np.nan

# 데이터 확인
print(df.head())

                 Date     City Product   Price Payment
0 2023-01-01 00:00:00    Seoul       A  2725.0    Cash
1 2023-01-01 01:00:00    Daegu       B  9929.0    Card
2 2023-01-01 02:00:00    Seoul       C  3118.0    Card
3 2023-01-01 03:00:00    Seoul       C  2901.0    Card
4 2023-01-01 04:00:00  Incheon       A  2739.0    Cash


In [ ]:
# 문제 1
fill_nan = df.groupby('City')['Price'].transform('median')

df['Price'] = df['Price'].fillna(fill_nan)
#print(df.info())
result = df[df['Payment']=='Card']['Price'].mean()
print(round(result, 2))

5426.29


In [ ]:
# 문제 2
df['Weekday'] = df['Date'].dt.weekday
q1 = df['Weekday'].isin([5, 6])
result = df.groupby('City')['Price'].sum().sort_values(ascending=False)
print(result)
# loc, iloc, isin 쓰는법


City
Incheon    815941.0
Busan      647059.5
Daegu      641636.0
Seoul      599213.0
Name: Price, dtype: float64


In [43]:
def get_category(price):
    if price < 4000: return 'Low'
    elif price < 7000: return 'Mid'
    else: return 'High'

df['Category'] = df['Price'].apply(get_category)
result = len(df[(df['Category']=='Mid') & (df['Product']=='A')])
print(result)

50
